Multiclass Classification 'Target'

In [4]:
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch

In [6]:
import pandas as pd
url = 'https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/raw/refs/heads/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.3.csv'
df = pd.read_csv(url)
df.head(5)

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
4,5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [7]:
df1 = df[['text','target','split']]
df1.head(10)

,text,target,split
0,dalits are lowlives,notgiven,train
1,gay people are a burden to our society,notgiven,train
2,Arabs are not welcome,notgiven,train
3,I'm not saying we should actually eliminate he...,notgiven,train
4,bananas are for black people,notgiven,test
5,asian people are the shit,none,test
6,women can not reproduce kids but men can in al...,notgiven,test
7,i hate wearing black in the summer!,none,test
8,Who cares what Chinese people think?,notgiven,test
9,i try to punch a letter box when i see one,notgiven,test


In [8]:
df1['target'] = pd.Categorical(df1['target']).codes
df1['target'] = df1['target'] - df1['target'].min()

C:\Users\Intel6\AppData\Local\Temp\ipykernel_6200\3734746050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target'] = pd.Categorical(df1['target']).codes
C:\Users\Intel6\AppData\Local\Temp\ipykernel_6200\3734746050.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target'] = df1['target'] - df1['target'].min()


In [9]:
print(df1['target'].unique())

[311 309 131 358 260  47 325 379 374 121 316 228 219 171  29  20 251 169
 194 179   8 207 190   3 202 188 321 302  17 310 304  43  28   1 312 206
 148 119 265 288   0  45 301  51  46  70  27 324 227  76 250 176  95 318
 125 262   9  36 168 332   5 167 308 315 406 183 338 259 122  31 314 166
 157 326 328 329 103 102 146 297 141 137 135 256 389 349 266  66  53 160
 200   4 133  74 317 110 191 408 340 313  39 161 350 154 348 248 231 384
 373 255 393 286 233  79 217 268 334 142 247 278 392  59  64  63  73 258
 239 218 216  52  93 257 149  50 177 295 404 232 132  60  15 294  75 173
  38 230  99 215 186 205 298  32 385 401  82  83  92 372 344 407 400 391
 107 371 184 225 150 221 130 269 357 336 229 124  88 155 353  30 339 252
  67  86 145  87 213 396 354 170   6 235 290 127 381 172 327 163 356  69
 101  49 195  81 152  62  12 253 203 335 104  23 337 386 126 274 129 390
  91  61 319 394 330 197 289 159 118 237 399 111 320 116 395  58 189 276
 405 164 296 291 144 403 242 261  25 264 347 199 27

In [10]:
df1[df1['target'] == 0]

,text,target,split
2220,I swear I will piss on this gross pikey until ...,0,train
2741,Really? There are a lot of gypsy travellers th...,0,train
2971,alot ogf people make fun of my personality in ...,0,train
4175,"I have worse teeth than a camel humper, my mum...",0,train
9093,Please stop inviting this squaw at the workplace,0,train
9185,A girl I'm working with hails from Egypt and t...,0,train
11812,fucking hell I've just seen a pikey,0,dev
23651,The purpose of the Aryan Hall is not just to h...,0,train


In [11]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Example data
texts = df1['text']
labels = df1['target'] # Categorical labels
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128

dataset = TextDataset(texts, labels, tokenizer, max_len)


In [12]:
train_texts = df1[df1['split'] == 'train']['text'].tolist()
train_labels = df1[df1['split'] == 'train']['target'].tolist()
val_texts = df1[df1['split'] == 'dev']['text'].tolist()
val_labels = df1[df1['split'] == 'dev']['target'].tolist()
test_texts = df1[df1['split'] == 'test']['text'].tolist()
test_labels = df1[df1['split'] == 'test']['target'].tolist()

# Assuming TextDataset is your custom dataset class
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, max_len)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_len)

In [16]:
num_labels = len(pd.Categorical(df1['target']).categories)
print(num_labels)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

410


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

  0%|          | 10/12348 [00:13<4:26:12,  1.29s/it]

{'loss': 5.9533, 'grad_norm': 13.803159713745117, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/12348 [00:26<4:25:41,  1.29s/it]

{'loss': 5.9663, 'grad_norm': 10.184335708618164, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 30/12348 [00:39<4:27:52,  1.30s/it]

...

100%|█████████▉| 12340/12348 [4:42:13<00:10,  1.35s/it]

{'loss': 0.4496, 'grad_norm': 20.017152786254883, 'learning_rate': 3.3760972316002705e-08, 'epoch': 3.0}


100%|██████████| 12348/12348 [4:42:24<00:00,  1.37s/it]

{'train_runtime': 16944.5661, 'train_samples_per_second': 5.829, 'train_steps_per_second': 0.729, 'train_loss': 1.019124189800223, 'epoch': 3.0}


TrainOutput(global_step=12348, training_loss=1.019124189800223, metrics={'train_runtime': 16944.5661, 'train_samples_per_second': 5.829, 'train_steps_per_second': 0.729, 'total_flos': 6520801542838272.0, 'train_loss': 1.019124189800223, 'epoch': 3.0})

In [30]:
print(df1['target'].unique())

[310 308 130 357 259  46 324 378 373 120 315 227 218 170  28  19 250 168
 193 178   7 206 189   2 201 187 320 301  16 309 303  42  27   0 311 205
 147 118 264 287 409  44 300  50  45  69  26 323 226  75 249 175  94 317
 124 261   8  35 167 331   4 166 307 314 405 182 337 258 121  30 313 165
 156 325 327 328 102 101 145 296 140 136 134 255 388 348 265  65  52 159
 199   3 132  73 316 109 190 407 339 312  38 160 349 153 347 247 230 383
 372 254 392 285 232  78 216 267 333 141 246 277 391  58  63  62  72 257
 238 217 215  51  92 256 148  49 176 294 403 231 131  59  14 293  74 172
  37 229  98 214 185 204 297  31 384 400  81  82  91 371 343 406 399 390
 106 370 183 224 149 220 129 268 356 335 228 123  87 154 352  29 338 251
  66  85 144  86 212 395 353 169   5 234 289 126 380 171 326 162 355  68
 100  48 194  80 151  61  11 252 202 334 103  22 336 385 125 273 128 389
  90  60 318 393 329 196 288 158 117 236 398 110 319 115 394  57 188 275
 404 163 295 290 143 402 241 260  24 263 346 198 27

Replaced -1 with 409

In [26]:
df1['target'] = df1['target'].replace(-1, 409)

C:\Users\Intel6\AppData\Local\Temp\ipykernel_9984\1551034730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target'] = df1['target'].replace(-1, 409)


In [33]:
df1[df['target'] == 409]

,text,target,split


In [34]:
invalid_rows = df1[df1['target'] == 409]
print(invalid_rows)

                                                    text  target  split
2220   I swear I will piss on this gross pikey until ...     409  train
2741   Really? There are a lot of gypsy travellers th...     409  train
2971   alot ogf people make fun of my personality in ...     409  train
4175   I have worse teeth than a camel humper, my mum...     409  train
9093    Please stop inviting this squaw at the workplace     409  train
9185   A girl I'm working with hails from Egypt and t...     409  train
11812                fucking hell I've just seen a pikey     409    dev
23651  The purpose of the Aryan Hall is not just to h...     409  train


In [18]:
model.save_pretrained('./results')

In [19]:
tokenizer.save_pretrained('./results')

('./results\\tokenizer_config.json',
 './results\\special_tokens_map.json',
 './results\\vocab.txt',
 './results\\added_tokens.json')

In [20]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('./results')

In [31]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('./results')

# Prepare your input text
text = "fucking hell"
inputs = tokenizer(text, return_tensors='pt')

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)
category_mapping = dict(enumerate(pd.Categorical(df['target']).categories))

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=1).item()

print(f"Predicted class: {predicted_class} \n Targetted group: {category_mapping[predicted_class]}")

Predicted class: 309 
 Targetted group: notargetrecorded


In [14]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the trained model
model = BertForSequenceClassification.from_pretrained('./results')
model.eval()

# Create DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

def evaluate_model(model, dataloader):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            outputs = model(input_ids=inputs, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return all_preds, all_labels

def test_model():
    preds, labels = evaluate_model(model, test_loader)
    macro_f1 = f1_score(labels, preds, average='macro')
    print(f'Macro F1 Score: {macro_f1}')
    
test_model()

Macro F1 Score: 0.135716318197409


### Results

In this section, we present the performance of our models and compare them with previous work where available. The primary evaluation metric used is the **macro F1-score**, which balances precision and recall across all classes, making it suitable for datasets with class imbalance.

#### Binary Classification

For the binary classification task of detecting hate speech ('hate' vs. 'nothate'), we compared our fine-tuned BERT model with the RoBERTa model reported in the DHate paper. The results are as follows:

##### Table 1: Binary Classification Results (Macro F1-score)

| **Model**              | **Method**   | **Macro F1-Score** |
|------------------------|--------------|--------------------|
| RoBERTa (DHate paper)  | Fine-tuned   |       0.7538       |
| **Our BERT Model**     | Fine-tuned   |     **0.7640**     |

As shown in Table 1, our fine-tuned BERT model achieved a macro F1-score of **0.7640**, outperforming the RoBERTa model from the DHate paper, which achieved a macro F1-score of **0.7538**. This improvement suggests that our model effectively captures the nuances of hate speech detection in the binary classification setting.

#### Multiclass Classification

For the multiclass classification tasks, we focused on two aspects:

1. **Target Classification**: Identifying the target group of the hate speech.
2. **Type Classification**: Determining the type of hate speech expressed.

To the best of our knowledge, there is no prior work that has performed multiclass classification in the domain of hate speech detection using these specific categories. Therefore, we present our results as a baseline for future research.

##### Table 2: Multiclass Classification Results (Macro F1-score)

| **Task**                | **Our BERT Model (Macro F1-Score)** |
|-------------------------|-------------------------------------|
| Target Classification   |               0.135                 |
| Type Classification     |               0.475                 |

In Table 2, the macro F1-score for **target classification** is **0.135**, indicating that the model has difficulty distinguishing between different target groups. This could be due to class imbalance or subtle differences between classes that are hard for the model to learn.

For **type classification**, the model achieved a macro F1-score of **0.475**, showing better performance compared to the target classification task. This suggests that the model is more effective at identifying the type of hate speech than pinpointing the specific target group.

#### Analysis

The superior performance of our BERT model in binary classification compared to the RoBERTa model may be attributed to several factors:

- **Data Preprocessing**: Our meticulous data cleaning and preprocessing steps might have improved the quality of input data.
- **Fine-Tuning Strategies**: The hyperparameter tuning and training strategies employed could have led to better model generalization.
- **Model Architecture**: While both BERT and RoBERTa are transformer-based models, differences in architecture and training objectives can impact performance.

For the multiclass tasks, the lower macro F1-score in target classification highlights the challenge in detecting specific target groups within hate speech. This opens avenues for future work to improve model performance, possibly by employing techniques to handle class imbalance or incorporating external knowledge sources.

### Conclusion

Our experiments demonstrate that the fine-tuned BERT model performs effectively in binary hate speech detection, surpassing the RoBERTa model from the DHate paper. In multiclass classification tasks, our results provide a baseline for target and type classification within hate speech detection. Future research can build upon these findings to enhance model performance, especially in distinguishing between various target groups.